In [2]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline

In [3]:
farmgate = pd.read_csv('../data/raw/FarmgatePrice.csv')
retail = pd.read_csv('../data/raw/RetailPrice.csv')
wholesale = pd.read_csv('../data/raw/WholesalePrice.csv')
production = pd.read_csv('../data/raw/VolumeOfProductionByMetricTons.csv')
harvested = pd.read_csv('../data/raw/AreaHarvestedInHectares.csv')
cpi = pd.read_csv('../data/raw/ConsumerPriceIndex.csv')

def melt_df(df, column_1, column_2, column_value):
    df = df.copy()
    
    df[column_1] = df[column_1].str.lower().str.strip()
    df[column_2] = df[column_2].str.lower().str.strip()
    
    df_long = df.melt(id_vars=[column_1, column_2], var_name='Date', value_name=column_value)
    df_long = df_long.rename(columns={column_1: "Geolocation", column_2: "Commodity"})
    
    df_long.replace('..', pd.NA, inplace=True)
    df_long["Geolocation"] = df_long["Geolocation"].str.replace(r'^\.\.', '', regex=True).str.strip()
    
    df_long[column_value] = pd.to_numeric(df_long[column_value], errors='coerce')
    
    return df_long

farmgate_copy = melt_df(farmgate, "Geolocation", "Commodity", "Price")
retail_copy = melt_df(retail, "Geolocation", "Commodity", "Price")
wholesale_copy = melt_df(wholesale, "Geolocation", "Commodity", "Price")
cpi_copy = melt_df(cpi, "Geolocation", "Commodity Description", "Consumer_Price_Index")

In [4]:
def to_datetime(df, is_cpi = False):
    df = df.copy()
    
    if is_cpi:
        df['Date'] = pd.to_datetime(df['Date'], format='%Y %b')
    else:
        df['Date'] = pd.to_datetime(df['Date'], format='%Y %B')
        
    df = df.sort_values(['Geolocation', 'Commodity', 'Date'])
    
    return df

def distribute_quarterly_to_monthly(df, column_1, column_2, value_column_name):
    df = df.copy()
    
    # Melt dataset to long format
    df_long = df.melt(id_vars=[column_1, column_2], var_name="Quarter", value_name=value_column_name)
    
    # Extract year and quarter from the Quarter column
    df_long["Year"] = df_long["Quarter"].str[:4].astype(int)
    df_long["Quarter"] = df_long["Quarter"].str[-1].astype(int)

    quarter_to_months = {
        1: [1, 2, 3],
        2: [4, 5, 6],
        3: [7, 8, 9],
        4: [10, 11, 12]
    }
    
    expanded_rows = []
    
    for _, row in df_long.iterrows():
        for month in quarter_to_months[row["Quarter"]]:
            expanded_rows.append({
                column_1: row[column_1],
                column_2: row[column_2],
                "Date": pd.Timestamp(year=row["Year"], month=month, day=1),
                value_column_name: row[value_column_name] / 3
            })
            
    df_monthly = pd.DataFrame(expanded_rows)
    df_monthly = df_monthly.rename(columns={column_1: "Geolocation", column_2: "Commodity"})
    
    return df_monthly

production_copy = distribute_quarterly_to_monthly(production, "Geolocation", "Ecosystem/Croptype", "Production_Volume_in_MetricTons")
harvested_copy = distribute_quarterly_to_monthly(harvested, "Geolocation", "Ecosystem/Croptype", "Area_Harvested_in_Hectares")
        
farmgate_copy = to_datetime(farmgate_copy)
wholesale_copy = to_datetime(wholesale_copy)
retail_copy = to_datetime(retail_copy)
cpi_copy = to_datetime(cpi_copy, is_cpi=True)
production_copy = to_datetime(production_copy)
harvested_copy = to_datetime(harvested_copy)


In [5]:
REGIONS = {
    "autonomous region in muslim mindanao (armm)" : "bangsamoro autonomous region in muslim mindanao (barmm)",
    "bangsamoro autonomous region in muslim mindanao (barmm)" : "bangsamoro autonomous region in muslim mindanao (barmm)",
    "cordillera administrative region (car)" : "cordillera administrative region (car)",
    "mimaropa region" : "mimaropa region",
    "region i (ilocos region)" : "region i (ilocos region)",
    "region ii (cagayan valley)" : "region ii (cagayan valley)",
    "region iii (central luzon)" : "region iii (central luzon)",
    "region iv-a (calabarzon)" : "region iv-a (calabarzon)",
    "region ix (zamboanga peninsula)" : "region ix (zamboanga peninsula)",
    "region v (bicol region)" : "region v (bicol region)",
    "region vi (western visayas)" : "region vi (western visayas)",
    "region v (bicol region)" : "region vi (western visayas)",
    "region vii (central visayas)" : "region vii (central visayas)",
    "region viii (eastern visayas)" : "region viii (eastern visayas)",
    "region ix (zamboanga peninsula)" : "region x (northern mindanao)",
    "region x (northern mindanao)" : "region x (northern mindanao)",
    "region xi (davao region)" : "region xi (davao region)",
    "region xii (soccsksargen)" : "region xii (soccsksargen)",
    "region xiii (caraga)" : "region xiii (caraga)"
}

def region_mapping(df, column_name, region_mapping):
    df = df.copy()
    
    df[column_name] = df[column_name].map(region_mapping)
    
    return df

farmgate_copy = region_mapping(farmgate_copy, 'Geolocation', REGIONS)
wholesale_copy = region_mapping(wholesale_copy, 'Geolocation', REGIONS)
retail_copy = region_mapping(retail_copy, 'Geolocation', REGIONS)
production_copy = region_mapping(production_copy, 'Geolocation', REGIONS)
harvested_copy = region_mapping(harvested_copy, 'Geolocation', REGIONS)
cpi_copy = region_mapping(cpi_copy, 'Geolocation', REGIONS)

datasets = [farmgate_copy, retail_copy, wholesale_copy, production_copy, harvested_copy, cpi_copy]

In [6]:
def convert_data(df, column_value):
    df = df.copy()

    DATES = [date.strftime('%Y-%m-%d') for date in sorted(pd.to_datetime(df['Date'].unique()))]

    farmgate_columns = {f'{i}': [] for i in ['Geolocation', 'Commodity'] + DATES}

    for Region in tqdm(df['Geolocation'].unique(), desc="Processing Regions"):
        for Commodity in tqdm(df['Commodity'].unique(), desc="Processing Commodities", leave=False):
            farmgate_columns['Geolocation'].append(Region)
            farmgate_columns['Commodity'].append(Commodity)
            
            for Date in tqdm(df['Date'].unique(), desc="Processing Dates", leave=False):
                Value = df[
                    (df['Geolocation'] == Region) & 
                    (df['Commodity'] == Commodity) & 
                    (df['Date'] == Date)
                ][column_value]            
                farmgate_columns[Date.strftime("%Y-%m-%d")].append(Value.sum() if not Value.empty else 0)
                
                
    # Print the final dictionary
    farmgate_transform_result = pd.DataFrame(farmgate_columns)
    
    return farmgate_transform_result

In [7]:
farmgate_converted = convert_data(farmgate_copy, "Price")
wholesale_converted = convert_data(wholesale_copy, "Price")
retail_converted = convert_data(wholesale_copy, "Price")
production_converted = convert_data(production_copy, "Production_Volume_in_MetricTons")
harvested_converted = convert_data(harvested_copy, "Area_Harvested_in_Hectares")
cpi_converted = convert_data(cpi_copy, "Consumer_Price_Index")

Processing Regions:   0%|          | 0/14 [00:00<?, ?it/s]

Processing Commodities:   0%|          | 0/6 [00:00<?, ?it/s]

Processing Dates:   0%|          | 0/60 [00:00<?, ?it/s]

Processing Dates:   0%|          | 0/60 [00:00<?, ?it/s]

Processing Dates:   0%|          | 0/60 [00:00<?, ?it/s]

...

Processing Dates:   0%|          | 0/60 [00:00<?, ?it/s]

Processing Dates:   0%|          | 0/60 [00:00<?, ?it/s]

Processing Commodities:   0%|          | 0/2 [00:00<?, ?it/s]

Processing Dates:   0%|          | 0/60 [00:00<?, ?it/s]

Processing Dates:   0%|          | 0/60 [00:00<?, ?it/s]

In [8]:
final_df = pd.concat([farmgate_converted, wholesale_converted, retail_converted, production_converted, harvested_converted, cpi_converted])

final_df.to_csv('../data/final/final_df.csv', index=False)